In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import spacy
import string
import gensim
import operator
import re

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
data = pd.read_csv('vehicle.csv')

In [3]:
data.head()

,No,Buy/sell,Condition,Brand,Model,Type,Color,Year,Keywords
0,1,buy,brandnew,toyota,corolla,car,black,2008,buy brandnew toyota corolla car black 2008
1,2,buy,brandnew,bmw,x1,suv,white,2007,buy brandnew bmw x1 suv white 2007
2,3,buy,recondition,benz,c200,car,green,2017,buy recondition benz c200 car green
3,4,buy,brandnew,audi,a1,car,ash,2007,buy brandnew audi a1 car ash 2007
4,5,sell,brandnew,bmw,x3,suv,black,2007,sell brandnew bmw x3 suv black 2007


In [4]:
#describe the unique values in categorical columns
categorical_col = []
for col in data.columns:
    if data[col].dtype == object:
        categorical_col.append(col)
        print(f"{col} => {data[col].unique()}")

Buy/sell => ['buy   ' 'buy' 'buy ' 'sell' 'sell   ' 'buy  ']
Condition => ['brandnew' 'recondition']
Brand => ['toyota' 'bmw' 'benz' 'audi']
Model => ['corolla' 'x1' 'c200' 'a1' 'x3' 'x2' 'allion' 'c180' 'a3' 'yaris' 'KDH']
Type => ['car' 'suv' 'van']
Color => ['black' 'white' 'green' 'ash' 'red' 'blue']
Keywords => ['buy    brandnew toyota corolla car black 2008'
 'buy brandnew bmw x1 suv white 2007'
 'buy recondition benz c200 car green' 'buy brandnew audi a1 car ash 2007'
 'sell brandnew bmw x3 suv black 2007'
 'buy    recondition bmw x2 suv red 2007'
 'sell    recondition toyota allion car white 2007'
 'sell brandnew benz c180 car ash 2014'
 'buy brandnew audi a3 car black 2007'
 'buy brandnew toyota yaris car red 2018'
 'sell brandnew audi a1 car green 2007'
 'sell    recondition benz c180 car black 2012'
 'buy brandnew benz c200 car white 2007'
 'buy   recondition audi a3 car green 2008'
 'sell    recondition toyota corolla car red 2008'
 'buy brandnew benz c180 car ash 2012'
 's

In [5]:
from sklearn.preprocessing import LabelEncoder
#handling categorical variable through label encoding
label_encoder = LabelEncoder()
for col in categorical_col:
    data[col] = label_encoder.fit_transform(data[col])

In [6]:
data.head()

,No,Buy/sell,Condition,Brand,Model,Type,Color,Year,Keywords
0,1,3,0,3,6,0,1,2008,0
1,2,0,0,2,7,1,5,2007,39
2,3,1,1,1,5,0,3,2017,83
3,4,0,0,0,1,0,0,2007,5
4,5,4,0,2,9,1,1,2007,117


In [7]:
#splitting dataset into dependent and independent variables
X = data.drop(['No','Keywords'],axis=1)
y = data.Keywords

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Buy/sell   500 non-null    int32
 1   Condition  500 non-null    int32
 2   Brand      500 non-null    int32
 3   Model      500 non-null    int32
 4   Type       500 non-null    int32
 5   Color      500 non-null    int32
 6   Year       500 non-null    int64
dtypes: int32(6), int64(1)
memory usage: 15.8 KB


In [9]:
y.head()

0      0
1     39
2     83
3      5
4    117
Name: Keywords, dtype: int32

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)#,random_state=42)

model= DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test,predictions)
score

0.73

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('RFC', RandomForestClassifier()))

In [12]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


LR: 0.627500 (0.042500)


C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\discriminant_analysis.py", line 553, in fit
    self._solve_svd(X, y)
  File "C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\discriminant_analysis.py", line 478, in _solve_svd
    _, S, Vt = linalg.sv

LDA: nan (nan)
KNN: 0.672500 (0.050559)
CART: 0.810000 (0.043589)


C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


NB: 0.810000 (0.043589)
SVM: 0.725000 (0.038730)


C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


RFC: 0.810000 (0.043589)


In [13]:
key = ['LogisticRegression','KNeighborsClassifier','SVC','DecisionTreeClassifier','RandomForestClassifier']
value = [LogisticRegression(), KNeighborsClassifier(n_neighbors=2, weights = 'uniform'),SVC(kernel = "rbf", random_state=42),DecisionTreeClassifier(random_state=10), RandomForestClassifier(n_estimators=60, random_state=0)]
models = dict(zip(key,value))
models

{'LogisticRegression': LogisticRegression(),
 'KNeighborsClassifier': KNeighborsClassifier(n_neighbors=2),
 'SVC': SVC(random_state=42),
 'DecisionTreeClassifier': DecisionTreeClassifier(random_state=10),
 'RandomForestClassifier': RandomForestClassifier(n_estimators=60, random_state=0)}

In [14]:
predicted =[]
for name, algo in models.items():
    model=algo
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    acc = accuracy_score(y_test, predict)
    predicted.append(acc)
    print(name,acc)

LogisticRegression 0.6
KNeighborsClassifier 0.69
SVC 0.1
DecisionTreeClassifier 0.73
RandomForestClassifier 0.73


C:\Users\Ushana\anaconda3\envs\Semantic\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
model.predict([[0,0,2,7,1,5,2008]])

array([39])

In [17]:
'''
Loading Gensim and nltk libraries
'''
# !pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [18]:
stemmer = SnowballStemmer("english")

original_words = ['want','buy','corolla','toyota','bmw','audi','benz','black','green','i','sell','KDH','2008','2009','2007','2010','2011','2012','2103','2014','2015','2016','2017','2018','2019','2020','2021','brandnew','recondition','red','white','x1','c200','c180']

singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,want,want
1,buy,buy
2,corolla,corolla
3,toyota,toyota
4,bmw,bmw
5,audi,audi
6,benz,benz
7,black,black
8,green,green
9,i,i


In [19]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    
    return result

In [34]:
import tkinter as tk
from tkinter import simpledialog
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#win= tk.Tk()
#Set the geometry of tkinter frame
#ROOT.geometry("750x250")
ROOT = tk.Tk()

ROOT.withdraw()
# the input dialog
USER_INP = simpledialog.askstring(title="Test",
                                  prompt="What you want?:")

# check it out
print(USER_INP)

i want to buy brandnew toyota corolla black car which made in 2008


In [35]:
sentence = USER_INP
split_sentence = sentence.split(' ')
print(split_sentence)

['i', 'want', 'to', 'buy', 'brandnew', 'toyota', 'corolla', 'black', 'car', 'which', 'made', 'in', '2008']


In [36]:
arr_Length=len(split_sentence)
print(arr_Length)

13


In [37]:
i=0
keywords=[]
while i<arr_Length:
    def buySell(x):
        if x =='buy':
            keywords.append("0")
        elif x =='sell':
            keywords.append("1")
    def condition(x):
        if x =='brandnew':
            keywords.append("0")
        elif x =='recondition':
            keywords.append("1")
    def brand(x):
        if x =='toyota':
            keywords.append("3")
        elif x =='bmw':
            keywords.append("2")
        elif x =='benz':
            keywords.append("1")
        elif x =='audi':
            keywords.append("0")
    def model(x):
        if x =='corolla':
            keywords.append("6")
        elif x =='x1':
            keywords.append("7")
        elif x =='c200':
            keywords.append("5") 
        elif x =='a1':
            keywords.append("1")  
        elif x =='x3':
            keywords.append("9")  
        elif x =='x2':
            keywords.append("2")
        elif x =='allion':
            keywords.append("0") 
        elif x =='c180':
            keywords.append("3") 
        elif x =='a3':
            keywords.append("8")
        elif x =='yaris':
            keywords.append("4")  
        elif x =='KDH':
            keywords.append("10")    
    def color(x):
        if x =='black':
            keywords.append("1")
        elif x =='white':
            keywords.append("5")
        elif x =='green':
            keywords.append("3") 
        elif x =='ash':
            keywords.append("0")  
        elif x =='red':
            keywords.append("2")  
        elif x =='blue':
            keywords.append("4")    
    def types(x):
        if x =='car':
            keywords.append("0")
        elif x =='suv':
            keywords.append("1")
        elif x =='van':
            keywords.append("2")    
    def year(x):
        if x =='2007':
            keywords.append("2007")
        elif x =='2008':
            keywords.append("2008")
    buySell(split_sentence[i])
    condition(split_sentence[i])
    brand(split_sentence[i])
    model(split_sentence[i])
    types(split_sentence[i])
    color(split_sentence[i])
    year(split_sentence[i])
    i+=1
print (keywords)

['0', '0', '3', '6', '1', '0', '2008']


In [38]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
score=accuracy_score(y_test,predictions)
score

0.73

In [39]:
test=model.predict([keywords])
test

array([52])

In [40]:
prediction_test = label_encoder.inverse_transform(test)

In [41]:
prediction_test

array(['buy brandnew toyota corolla car black 2008'], dtype=object)